In [1]:
from langchain.chains import ConversationalRetrievalChain
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.callbacks.streaming_stdout_final_only import (
    FinalStreamingStdOutCallbackHandler,
)
from langchain.agents import AgentType
import os
from dotenv import load_dotenv
load_dotenv()

pinecone_key = os.getenv("PINECONE_KEY")
pinecone_env = os.getenv("PINECONE_ENV")
OPENAI_API_KEY = ""

In [2]:
import pinecone
pinecone.init(api_key=pinecone_key, environment=pinecone_env)
index = pinecone.Index('openai')

d:\Applications\miniconda3\envs\ML\lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
from langchain.vectorstores import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
vectordb = Pinecone(index, embedding_function=embeddings.embed_query, text_key="text")

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks.streaming_stdout_final_only import (
    FinalStreamingStdOutCallbackHandler,
)
chat = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    temperature=0,
    model_name = "gpt-3.5-turbo-16k",
    streaming=True,
    callbacks=[FinalStreamingStdOutCallbackHandler()],
)

In [5]:
from langchain.chains import RetrievalQA

retriever = RetrievalQA.from_chain_type(
    llm=chat,
    chain_type="stuff",
    retriever=vectordb.as_retriever(),
    
)

In [6]:
from langchain.agents import Tool

tool_desc = """Use this tool to answer user questions using AntStack's data. This tool can also be used for follow up questions from the user"""

tools = [Tool(
    func=retriever.run,
    description=tool_desc,
    name='AntStack DB'
)]

In [7]:
from langchain.memory import DynamoDBChatMessageHistory

chat_history = DynamoDBChatMessageHistory(table_name='conversation-store', session_id='qasc')

In [8]:
from langchain.chains.conversation.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)
memory.chat_memory = chat_history

In [58]:
from langchain.agents import initialize_agent
from langchain.agents import AgentType

conversational_agent = initialize_agent(
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION, 
    tools=tools, 
    llm=chat,
    verbose=False,
    memory=memory,
)

In [59]:
sys_msg = """You are a chatbot for a Serverless company AntStack and strictly answer the question based on the context below, and if the question can't be answered based on the context, say \"I'm sorry I cannot answer the question, contact connect@antstack.com\"
"""

prompt = conversational_agent.agent.create_prompt(
    system_message=sys_msg,
    tools=tools
)
conversational_agent.agent.llm_chain.prompt = prompt

In [ ]:
memory.chat_memory.messages

In [61]:
conversational_agent.run(
    input="What are you?"
)

'I am an AI-powered chatbot developed by AntStack. I am here to assist you with any questions or concerns you may have about our serverless services.'

In [66]:
hi = conversational_agent.run("What are you?")
print(hi)